In [3]:
import pandas as pd

# let's check the completeness of the model output
import os

folder_path = "resource/output/ICL"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        length = len(df)
        print(f"{filename} 的长度为: {length}")

exp_result_Llama-2-7b-chat-hf_20240519070513.csv 的长度为: 100
exp_result_Llama-2-7b-chat-hf_20240520114017.csv 的长度为: 6


In [5]:
result_complete = pd.read_csv(
    "/Users/lujun.li/projects/causallm-github/causalllm/resource/input/causal_discovery/exp_result_Llama-2-7b-chat-hf_20240506004401.csv"
)

In [14]:
result_complete["generated_response"].sample(1).to_dict()

{17596: "\n<s>[INST] <<SYS>>\nYou are a conscientious assistant, and you must answer my questions exactly as my questions require.\n<</SYS>>\nPlease reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query.\n\n    Examples: \n\n    Premise: A dog is chewing its bone.\n    Query: What would have happened if the dog had buried the bone?\n    Possible answers: Nothing would have happened.\n    Correct answer: The dog would have eaten the bone.\n    Premise: A dog is chewing its bone.\nQuery: What would have happened if the dog had buried the bone?\nPossible answers: Nothing would have happened.The dog would have eaten the bone.That is not possible.\nCorrect answer:\n[/INST]\nOf course, I'm happy to help! Here's the answer to your final query:\n\nPremise: A dog is chewing its bone.\nQuery: What would have happened if the dog had buried the bone?\nPossible answers: Nothing would have happened.The dog would have eaten t

In [22]:
def extract_final_response(row):
    prompt = row["llama2_chat_initial_prompt"]
    response = row["generated_response"]
    return response[len(prompt) :]


result_complete["generated_response_pure"] = result_complete.apply(
    extract_final_response, axis=1
)

In [31]:
result_complete["generated_response_pure"].sample(1).to_dict()

{597: "Of course, I'm happy to help! Here's how I would answer your final query:\n\nPremise: A train crashes into a mountain.\nQuery: What would have happened if the train had crashed into a feather?\nPossible answers: The train would have been fine.The train would have had a headache.The train would have been destroyed.\nCorrect answer: The train would have had a headache.\n\nExplanation:\n\nThe premise states that a train crashes into a mountain, which is a very dramatic and serious event. The query then asks what would have happened if the train had crashed into a feather, which is a very absurd and nonsensical scenario. The possible answers provided are a mix of serious and nonsensical options, but the correct answer is the only one that makes sense in this context. The train would have had a headache because a feather is a very soft and light object that would not have caused any significant damage to the train."}

In [120]:
import re


def find_correct_answer(sentence):
    pattern = r"Correct answer:(.+?)\."
    match = re.search(pattern, sentence)
    if match:
        return match.group(1), True
    else:
        return "", False


result_complete[["generated_answer", "is_answer_found"]] = result_complete[
    "generated_response_pure"
].apply(lambda x: pd.Series(find_correct_answer(x)))

In [121]:
# The model confused about the input by asking for the query

result_complete[result_complete["is_answer_found"] == False][
    "generated_response_pure"
].str.contains("Please provide")

4        False
18        True
23        True
24       False
25       False
         ...  
19169    False
19170    False
19172    False
19173    False
19174     True
Name: generated_response_pure, Length: 5201, dtype: bool

In [122]:
result_complete["is_confused"] = result_complete[
    result_complete["is_answer_found"] == False
]["generated_response_pure"].apply(lambda x: True if "Please provide" in x else False)

In [123]:
result_complete["is_confused"].fillna(False, inplace=True)

/var/folders/k6/w4zkqz352hzf1fs2pc1tqjf88vhly_/T/ipykernel_18029/3585300940.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_complete["is_confused"].fillna(False, inplace=True)
/var/folders/k6/w4zkqz352hzf1fs2pc1tqjf88vhly_/T/ipykernel_18029/3585300940.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

In [124]:
print("lengh of not detected correct answer but without confusion")
print(
    len(
        result_complete[
            (result_complete["is_confused"] == False)
            & (result_complete["is_answer_found"] == False)
        ]
    )
)

lengh of not detected correct answer but without confusion
1844


In [125]:
# label 1 : found correct answer
# label 2 : correct answer not found because they didn't unstand the examples and ask for query
# label 3 : didn't understand anything
result_complete.loc[result_complete["is_answer_found"] == True, "label"] = 1

result_complete.loc[
    (result_complete["is_answer_found"] == False)
    & (result_complete["is_confused"] == True),
    "label",
] = 2

result_complete.loc[
    (result_complete["is_answer_found"] == False)
    & (result_complete["is_confused"] == False),
    "label",
] = 3

In [163]:
import string


def remove_punctuation(text):
    return re.sub(r"[^\w\s]", "", text)


result_complete["CorrectAnswerCleaned"] = result_complete["CorrectAnswer"].str.replace(
    r"[^\w\s]", "", regex=True
)

result_complete["exact_match"] = (
    result_complete["generated_answer"].str.strip()
    == result_complete["CorrectAnswerCleaned"]
)
result_complete["exact_match"].value_counts()

exact_match
False    19011
True       169
Name: count, dtype: int64

In [175]:
result_complete[result_complete["Temperature"] == 0.1][
    [
        "generated_answer",
        "CorrectAnswer",
        "QCC",
        "Premise",
        "llama2_chat_initial_prompt",
    ]
].sample(1).to_dict()

{'generated_answer': {15556: ' The woman would not have become breathless'},
 'CorrectAnswer': {15556: 'The woman would have become breathless.'},
 'QCC': {15556: 'What would have happened if she had run away?'},
 'Premise': {15556: 'A woman does not run away.'},
 'llama2_chat_initial_prompt': {15556: '\n<s>[INST] <<SYS>>\nYou are a conscientious assistant, and you must answer my questions exactly as my questions require.\n<</SYS>>\nPlease reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query.\n\n    Examples: \n\n    Premise: A woman does not run away.\n    Query: What would have happened if she had run away?\n    Possible answers: The woman would have become breathless.\n    Correct answer: The woman would not have become breathless.\n    Premise: A woman does not run away.\nQuery: What would have happened if she had run away?\nPossible answers: The woman would have become breathless.The woman would not have 

In [176]:
from Levenshtein import distance


def levenshtein_similarity(str1, str2):
    max_length = max(len(str1), len(str2))
    if max_length == 0:
        return 1.0
    else:
        return 1.0 - (distance(str1, str2) / max_length)


result_complete["levenshtein_similarity"] = result_complete.apply(
    lambda row: levenshtein_similarity(
        row["generated_answer"].strip(), row["CorrectAnswerCleaned"].strip()
    ),
    axis=1,
)

In [183]:
result_complete[
    (result_complete["levenshtein_similarity"] >= 0.95)
    & (result_complete["levenshtein_similarity"] < 1)
][
    [
        "generated_answer",
        "CorrectAnswer",
        "QCC",
        "Premise",
        "llama2_chat_initial_prompt",
        "levenshtein_similarity",
    ]
]

,generated_answer,CorrectAnswer,QCC,Premise,llama2_chat_initial_prompt,levenshtein_similarity
5172,Being around her would have been unpleasant,Being around her would have been pleasant.,What would have happened if she had been quieter?,A woman is loud and annoying.,\n<s>[INST] <<SYS>>\nYou are a conscientious a...,0.953488
13392,Being around her would have been unpleasant,Being around her would have been pleasant.,What would have happened if she had been quieter?,A woman is loud and annoying.,\n<s>[INST] <<SYS>>\nYou are a conscientious a...,0.953488
17228,Being around her would have been unpleasant,Being around her would have been pleasant.,What would have happened if she had been quieter?,A woman is loud and annoying.,\n<s>[INST] <<SYS>>\nYou are a conscientious a...,0.953488
18872,Being around her would have been unpleasant,Being around her would have been pleasant.,What would have happened if she had been quieter?,A woman is loud and annoying.,\n<s>[INST] <<SYS>>\nYou are a conscientious a...,0.953488


In [187]:
result_complete.groupby()

,Unnamed: 0,Repetition_times,Temperature,PCTID,BatchID,Premise,QCC,CorrectAnswer,Answer1,Answer2,...,temperature,model,generated_response_pure,generated_answer,is_answer_found,is_confused,label,exact_match,levenshtein_similarity,CorrectAnswerCleaned
0,0,0,0.1,1,1,A woman opens a treasure chest.,What would have happened if the woman had not ...,The treasure chest would have remained closed.,The treasure chest would have been open.,That is not possible.,...,0.1,Llama-2-7b-chat-hf,"Of course, I'm happy to help! Here's how I wou...",The treasure chest would have been open,True,False,1.0,False,0.777778,The treasure chest would have remained closed
1,1,0,0.1,2,1,A police officer calms down a hostage-taker.,What would have happened if the police officer...,The hostages would have remained in danger.,That is not possible.,The hostage-taker would have released the host...,...,0.1,Llama-2-7b-chat-hf,"Of course, I'm happy to help! Here's how I wou...",That is not possible,True,False,1.0,False,0.214286,The hostages would have remained in danger
2,2,0,0.1,3,1,A man talks about a lion.,What would have happened if the man had talked...,"Without a barrier, the man would have been eaten.","Without a barrier, the lion would have been ea...",Nothing special would have happened.,...,0.1,Llama-2-7b-chat-hf,"Of course, I'm happy to help! Here's how I wou...","Without a barrier, the lion would have been e...",True,False,1.0,False,0.918367,Without a barrier the man would have been eaten
3,3,0,0.1,4,1,A girl kisses a boy.,What would have happened if the girl had slapp...,The girl would have been angry.,The girl would have been happy.,That is not possible.,...,0.1,Llama-2-7b-chat-hf,"Of course, I'm happy to help! Based on the thr...",The girl would have been happy,True,False,1.0,False,0.866667,The girl would have been angry
4,4,0,0.1,5,1,A girl kisses a boy.,What would have happened if the girl had kille...,She would have been liable to prosecution.,That is not possible.,The boy would have been arrested for assault.,...,0.1,Llama-2-7b-chat-hf,"I apologize, but I cannot provide answers to y...",,False,False,3.0,False,0.000000,She would have been liable to prosecution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19175,19175,9,1.9,270,5,A woman does not practice her moves.,What would have happened if she had practiced ...,The woman would have become more skilled.,The woman would not have become more skilled.,That is not possible.,...,1.9,Llama-2-7b-chat-hf,"Understood, kind manager! Here's the answer to...",The man would not have been in a better finan...,True,False,1.0,False,0.406780,The woman would have become more skilled
19176,19176,9,1.9,271,5,A woman does not prepare for someone's reception.,What would have happened if she had prepared it?,The woman would have put on make-up.,The woman would not have put on make-up.,That is not possible.,...,1.9,Llama-2-7b-chat-hf,Premise: A dog runs to the bakery.\n\nQuery: W...,The dog would not have purchased bread,True,False,1.0,False,0.473684,The woman would have put on makeup
19177,19177,9,1.9,272,5,A woman does not put pen to paper.,What would have happened if she had put pen to...,The woman would have moved her right hand.,The woman would not have moved her right hand.,That is not possible.,...,1.9,Llama-2-7b-chat-hf,Premise: A man does not walk backwards.\nQuery...,The man would not have moved his feet in the ...,True,False,1.0,False,0.460317,The woman would have moved her right hand
19178,19178,9,1.9,273,5,A woman does not realize something.,What would have happened if she had realized s...,The woman would have gotten a new idea.,The woman would not have gotten a new idea.,That is not possible.,...,1.9,Llama-2-7b-chat-hf,"Certainly, here is the query and the correct a...",The woman would not have gotten a new idea,True,False,1.0,False,0.904762,The woman would have gotten a new idea


In [188]:
exact_match_counts = (
    result_complete[result_complete["exact_match"] == True].groupby("Premise").size()
)

In [191]:
# Calculate the total number of correct answers (exact matches)
total_correct = result_complete["exact_match"].sum()

# Calculate the total number of questions
total_questions = len(result_complete)

# Calculate the T1 score
T1_score = total_correct / total_questions
T1_score

0.00881126173096976